In [2]:
import json
import pandas as pd
import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout
from ipywidgets import Button, Layout, jslink, IntText, IntSlider
from IPython.display import display, Javascript, Markdown, clear_output, HTML
from ipywidgets import GridspecLayout
from markdown_it import MarkdownIt

In [3]:
# Custom CSS to ensure text wraps properly
css = """
<style>
.custom-button {
    white-space: normal;
    font-size: 16px !important;
    font-weight: bold !important;
}
</style>
"""

In [4]:
# Custom CSS to ensure text wraps properly
header_css = """
<style>
.custom-header {
    white-space: normal;
    font-size: 18px !important;
    font-weight: bold !important;
    padding: 50px;
    color: #000000
}
</style>
"""

In [5]:
# Custom CSS to ensure text wraps properly
footer_css = """
<style>
.custom-footer {
    width: auto;
    align-item: flex-start !important; 
    padding-top: 70px;
    height: auto
}
</style>
"""

In [6]:
# Header widget
html_header = widgets.HTML(value="<h1>Remote Camera Decision Support Tool</h1>")

In [7]:
html_header.add_class("custom-header")
# Create a HTML widget to include the custom CSS
header_css_widget = widgets.HTML(value=header_css)

In [8]:
html_footer = widgets.HTML(value="""
<footer>
    <img src='footer.png' alt='Footer Image'>
</footer>
""")


In [9]:
html_footer.add_class("custom-footer")
# Create a HTML widget to include the custom CSS
footer_css_widget = widgets.HTML(value=footer_css)

In [10]:
# Image Widget

file = open("../logo.png", "rb")
image = file.read()

image_headline = widgets.Image(
                    value=image,
                    format='jpg',
                    width='200'
                )

label_headline = widgets.Label(
                    value='',
                    style={'description_width': 'initial'}
                )

# vbox_headline = widgets.HBox([image_headline, label_headline])

In [11]:
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto')) #width='max-content'

In [12]:
# Concept Library

concept_header = create_expanded_button('Concept Library', 'success')
concept_header.style.button_color = '#2D6EB4'
concept_header.style.text_color = 'black'

concept_option1 = create_expanded_button('option-1', 'success')
concept_option1 .style.button_color = '#97C3F2'
concept_option1 .style.text_color = 'black'

concept_option2 = create_expanded_button('option-2', 'success')
concept_option2.style.button_color = '#B4D5F8'
concept_option2.style.text_color = 'black'

concept_option3 = create_expanded_button('option-3', 'success')
concept_option3.style.button_color = '#B4D5F9'
concept_option3.style.text_color = 'black'

concept_grid = GridspecLayout(4, 41, grid_gap='5px', height='170px')
concept_grid[0, 1:41] = concept_header
concept_grid[1, 1:41] = concept_option1
concept_grid[2, 1:41] = concept_option2
concept_grid[3, 1:41] = concept_option3
# concept_grid

In [13]:
left_col = widgets.VBox(
    [image_headline, concept_grid],
    layout=widgets.Layout(
        width='auto',  # Set the width to 50% of the container or viewport
        height='auto',  # Set a fixed height
#         border_right='solid 4px grey',
        padding='10px',
#         margin='auto',  # Center the VBox horizontally
        align_items='flex-start',  # Center the items inside the VBox   "flex-start"
        justify_content='center'  # Center the items vertically inside the VBox
    )
)


# vbox_headline = widgets.VBox([image_headline, concept_grid])
# display(left_col) 

In [14]:
global stored_choices, path

stored_choices = {}
path = "../temp_storage.json"
back_path = "../backtracking.json"

In [15]:
with open(path, "r") as f:
    stored_choices = json.load(f)

In [16]:
with open(back_path, 'r') as f:
    back_list = json.load(f)

In [29]:
# Read the CSV file into a DataFrame

df = pd.read_csv('new_format.csv', low_memory=False)

In [30]:
# seperates the string based on ',' also removes the whitespace after comma & makes a list

def split_and_trim(s):
    return [item.strip() for item in s.split(',')]

In [32]:
# gets the question based on value of other column

ques1 = df.loc[df['question_code'] == 'surv_dur_mth_min', 'question_text'].iloc[0]
opt1 = df.loc[df['question_code'] == 'surv_dur_mth_min', 'q_option_label'].iloc[0]
opt_codes1 = df.loc[df['question_code'] == 'surv_dur_mth_min', 'q_option_code'].iloc[0]

opt1 = split_and_trim(opt1)
opt_codes1 = split_and_trim(opt_codes1)

ques2 = df.loc[df['question_code'] == 'surv_dur_mth_max', 'question_text'].iloc[0]
opt2 = df.loc[df['question_code'] == 'surv_dur_mth_max', 'q_option_label'].iloc[0]
opt_codes2 = df.loc[df['question_code'] == 'surv_dur_mth_max', 'q_option_code'].iloc[0]

opt2 = split_and_trim(opt2)
opt_codes2 = split_and_trim(opt_codes2)

In [35]:
# Factor widgets

question1 = create_expanded_button(ques1, 'success')
question1.style.button_color = 'lightblue'
question1.style.text_color = 'black'
# Add custom class to the button
question1.add_class('custom-button')

option1 = create_expanded_button(opt1[0], 'success')
option1.style.button_color = 'lightgreen'
option1.style.text_color = 'black'

option2 = create_expanded_button(opt1[1], 'success')
option2.style.button_color = 'green'
option2.style.text_color = 'black'


# Create a HTML widget to include the custom CSS
css_widget = widgets.HTML(value=css)

factor_grid1 = GridspecLayout(4, 12, grid_gap='3px', height='250px')
factor_grid1[1:2, 2:6] = question1
factor_grid1[0, 7:9] = option1
factor_grid1[2, 7:9] = option2


# Second factor grid
question2 = create_expanded_button(ques2, 'success')
question2.style.button_color = 'lightblue'
question2.style.text_color = 'black'
# Add custom class to the button
question2.add_class('custom-button')

option2_1 = create_expanded_button(opt2[0], 'success')
option2_1.style.button_color = 'lightgreen'
option2_1.style.text_color = 'black'

option2_2 = create_expanded_button(opt2[1], 'success')
option2_2.style.button_color = 'green'
option2_2.style.text_color = 'black'


# Create a HTML widget to include the custom CSS
css_widget = widgets.HTML(value=css)

factor_grid2 = GridspecLayout(4, 12, grid_gap='3px', height='250px')
factor_grid2[1:2, 2:6] = question2
factor_grid2[0, 7:9] = option2_1
factor_grid2[2, 7:9] = option2_2
# factor_grid

logical_integer_container = widgets.VBox(children=[factor_grid1, factor_grid2, css_widget], layout=widgets.Layout(
    align_items='center'  # Center align vertically
#     border='solid 2px black',
#     width = "70%"
#     justify_content='center'  # Center align horizontally
#     height='100vh'  # Set height to 100% of viewport height for vertical centering
))
# logical_integer_container

In [38]:
# integer widgets

# question_widget1 = create_expanded_button(ques2, 'success')
# question_widget1.style.button_color = 'lightblue'
# question_widget1.style.text_color = 'black'
# # Add custom class to the button
# question_widget1.add_class('custom-button')

options_widget1 = widgets.IntText(
                            value=0,  # Default value
                            description="Enter value",
                            disabled=False,
                            layout={'width': 'max-content'}
                        )

# Create a HTML widget to include the custom CSS
css_widget = widgets.HTML(value=css)

integer_grid1 = GridspecLayout(4, 12, grid_gap='3px', height='200px')
# integer_grid1[1:3, 0:7] = question2
integer_grid1[1:2, 8:12] = options_widget1

integer_container1 = widgets.VBox(children=[integer_grid1, css_widget], layout=widgets.Layout(
    align_items='center'  # Center align vertically
#     border='solid 2px black',
#     width = "70%"
#     justify_content='center'  # Center align horizontally
#     height='100vh'  # Set height to 100% of viewport height for vertical centering
))


# question_widget2 = create_expanded_button(ques2, 'success')
# question_widget2.style.button_color = 'lightblue'
# question_widget2.style.text_color = 'black'
# # Add custom class to the button
# question_widget2.add_class('custom-button')

options_widget2 = widgets.IntText(
                            value=0,  # Default value
                            description="Enter value",
                            disabled=False,
                            layout={'width': 'max-content'}
                        )

# Create a HTML widget to include the custom CSS
css_widget = widgets.HTML(value=css)

integer_grid2 = GridspecLayout(4, 12, grid_gap='3px', height='200px')
# integer_grid2[1:3, 0:7] = question2
integer_grid2[1:2, 8:12] = options_widget2

# factor_grid

integer_container2 = widgets.VBox(children=[integer_grid2, css_widget], layout=widgets.Layout(
    align_items='center'  # Center align vertically
#     border='solid 2px black',
#     width = "70%"
#     justify_content='center'  # Center align horizontally
#     height='100vh'  # Set height to 100% of viewport height for vertical centering
))
# integer_container

In [23]:
output = widgets.Output()
def option_11_button_clicked(event):
    with output:
        clear_output()
        global selected_objective
        
        display(integer_container1)
        selected_objective = stored_choices['objective']
        
factor_grid1[0, 7:9].on_click(option_11_button_clicked)

In [24]:
output = widgets.Output()
def option_12_button_clicked(event):
    with output:
        clear_output()
        global selected_objective
        
        selected_option = factor_grid1[2, 7:9].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        stored_choices["surv_dur_mth_min"] = selected_option_code
        
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
        selected_objective = stored_choices['objective']
        
factor_grid1[2, 7:9].on_click(option_12_button_clicked)

In [39]:
output = widgets.Output()
def option_21_button_clicked(event):
    with output:
        clear_output()
        global selected_objective
        
        display(integer_container2)
        selected_objective = stored_choices['objective']
        
factor_grid2[0, 7:9].on_click(option_21_button_clicked)

In [40]:
output = widgets.Output()
def option_22_button_clicked(event):
    with output:
        clear_output()
        global selected_objective
        
        selected_option = factor_grid2[2, 7:9].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        stored_choices["surv_dur_mth_max"] = selected_option_code
        
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
        selected_objective = stored_choices['objective']
        
factor_grid2[2, 7:9].on_click(option_22_button_clicked)

In [41]:
# Define the iframe HTML for a YouTube video
iframe = """
<iframe width="800" height="500" src="https://ab-rcsc.github.io/RCSC-WildCAM_Remote-Camera-Survey-Guidelines-and-Metadata-Standards/2_metadata-standards/2_8.0_Deployment.html#deployment-start-date-time-dd-mmm-yyyy-hh-mm-ss" 
        frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" 
        allowfullscreen></iframe>
"""

# Display the iframe
# display(HTML(iframe))


In [42]:
# Create an Output widget for the pop-up window content
output_popup = widgets.Output()

# Function to show the pop-up window
def show_popup(b):
    with output_popup:
        output_popup.clear_output()
        display(popup_window)

# Function to close the pop-up window
def close_popup(b):
    output_popup.clear_output()

# Create a close button for the pop-up window
close_button = widgets.Button(
    description="Close",
    layout=widgets.Layout(width='100px')
)
close_button.on_click(close_popup)

In [43]:
# Create the pop-up window
popup_window = widgets.VBox(
    children=[
        widgets.HTML(iframe),
#         widgets.HTML('<h3>Pop-up Window</h3>'),
        close_button
    ],
    layout=widgets.Layout(
        display='flex',
        flex_flow='column',
        align_items='center',
        justify_content='center',
        border='solid 1px grey',
        padding='10px',
        width='auto',
        height='auto',
        background_color='lightyellow'
    )
)

In [44]:
# Create a button that will be positioned at the top right corner
button = widgets.Button(
    description="Show Popup"
#     layout=widgets.Layout(width='100px')
)
button.on_click(show_popup)

In [45]:
# Create a buttons for submitting the question

submit_button = widgets.Button(description="Submit")

In [46]:
# Create a button for going back to previous question

back_button = widgets.Button(description="Back")

In [47]:
# Create a button for skipping the question
skip_button = widgets.Button(description="Skip")
# skip_button.add_class("lbl_bg");

In [48]:
# For going back to last question

output = widgets.Output()
def on_back_button_clicked(event):
    with output:
        clear_output()
        file_name = back_list[-1]
        notebook_filename = file_name + ".ipynb"
        link = Markdown('''<a href="{}" target="_self" rel="noopener noreferrer">Previous</a>'''.format(notebook_filename))
        back_list.pop()
        with open (back_path, "w")as f:
            json.dump(back_list, f)
        display(link)

back_button.on_click(on_back_button_clicked)

In [49]:
output = widgets.Output()
def on_skip_button_clicked(event):
    with output:
        clear_output()
        link = Markdown('''<a href="https://voila.readthedocs.io/en/stable/deploy.html" target="_self"> 
                                    Next 
                            </a>''')
        display(link)

skip_button.on_click(on_skip_button_clicked)
# vbox_result = widgets.VBox([question_widget1, options_widget1, submit_button, output, choice])

In [50]:
output = widgets.Output()
def on_button_clicked(event):
    with output:
        clear_output()
        global selected_option1
        next_notebook = ''
        link = "Submit your answer for next question"

        condition = df.loc[df['page_name'] == 'sp_asymptote', 'json_logic'].iloc[0]
        condition_json = json.loads(condition)
        if selected_objective in condition_json['objective']:
            next_notebook = "sp_asymptote"
        else:
            next_notebook = "obj_targ_sp"
            
        notebook_filename = next_notebook + ".ipynb"
        
        link = Markdown('''<a href="{}" target="_self" rel="noopener noreferrer">Next</a>'''.format(notebook_filename))

        back_list.append("cam_strat_covar")
        with open (back_path, "w")as f:
            json.dump(back_list, f)
         
        if options_widget1.value > 0:
            stored_choices["surv_dur_mth_min"] = options_widget1.value
            options_widget1.value = 0
        
        
        if options_widget2.value > 0:
            stored_choices["surv_dur_mth_max"] = options_widget2.value
            options_widget2.value = 0
        
        with open(path, "w") as f:
            json.dump(stored_choices, f)

        display(link)

submit_button.on_click(on_button_clicked)

In [51]:

all_button = widgets.HBox(
    [back_button, submit_button, skip_button],
    layout=widgets.Layout(
        width='70%',  # Set the width to 50% of the container or viewport
        height='auto',  # Set a fixed height
#         border='solid 2px grey',
        padding='10px',
        margin='auto',  # Center the VBox horizontally
        align_items='center',  # Center the items inside the VBox
        justify_content='center'  # Center the items vertically inside the VBox
    )
)
buttons = widgets.VBox(
    [output, all_button],
    layout=widgets.Layout(
        width='70%',  # Set the width to 50% of the container or viewport
        height='auto',  # Set a fixed height
#         border='solid 2px grey',
        padding='10px',
        margin='auto',  # Center the VBox horizontally
        align_items='center',  # Center the items inside the VBox
        justify_content='center'  # Center the items vertically inside the VBox
    )
)



upper = widgets.VBox(
    [logical_integer_container, buttons],
    layout=widgets.Layout(
        width='70%',  # Set the width to 50% of the container or viewport
        height='auto',  # Set a fixed height
        border='solid 2px grey',
        padding='10px',
        margin='auto',  # Center the VBox horizontally
        align_items='center',  # Center the items inside the VBox
        justify_content='center'  # Center the items vertically inside the VBox
    )
)


popup = widgets.VBox(children=[button, output_popup], layout=widgets.Layout(
        width='70%',  # Set the width to 50% of the container or viewport
        height='auto',  # Set a fixed height
#         border='solid 4px grey',
        padding='10px',
        margin='auto',  # Center the VBox horizontally
        align_items='center',  # Center the items inside the VBox
        justify_content='center'  # Center the items vertically inside the VBox
    ))

right_col = widgets.VBox(children=[html_header, header_css_widget, upper, popup, html_footer, footer_css_widget], layout=widgets.Layout(
        width='70%',  # Set the width to 50% of the container or viewport
        height='auto',  # Set a fixed height
        border_left='solid 5px grey',
        padding='10px',
        margin='auto',  # Center the VBox horizontally
        align_items='center',  # Center the items inside the VBox
        justify_content='center'
))

cols = widgets.HBox(
    [left_col, right_col],
    layout=widgets.Layout(
        width='auto',  # Set the width to 50% of the container or viewport
        height='auto',  # Set a fixed height
        border='solid 4px grey',
#         padding='10px',
#         margin='auto',  # Center the VBox horizontally
        align_items='flex-start',  # Center the items inside the VBox
        justify_content='center',  # Center the items vertically inside the VBox
        background_color='lightyellow'
    )
)

# Display the container

# right_col.add_class("box_style")
# display(left_col)
# display(right_col)
display(cols)
# display(popup)